In [67]:
import pandas as pd
import numpy as np

In [33]:
from ast import literal_eval

In [3]:
movies = pd.read_csv('data/movies_metadata.csv', low_memory=False)

In [52]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [71]:
#movies

In [51]:
movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [72]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,english
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1


In [69]:
movies['english'] = movies['original_language'].apply(lambda x: 1 if x =='en' else 0)

In [75]:
movies_df = movies[['id', 'runtime', 'english']].copy()

In [74]:
movies_df

,id,runtime,english
0,862,81.0,1
1,8844,104.0,1
2,15602,101.0,1
3,31357,127.0,1
4,11862,106.0,1
...,...,...,...
45461,439050,90.0,0
45462,111109,360.0,0
45463,67758,90.0,1
45464,227506,87.0,1


In [53]:
keywords = pd.read_csv('data/keywords.csv')

In [56]:
keywords['keywords_'] = keywords['keywords'].apply(literal_eval)

In [59]:
keywords['keywords__'] = keywords['keywords_'].apply(lambda x: [d['name'].replace(" ", "") for d in x[0:4]])

In [76]:
keywords_df = keywords[['id', 'keywords__']].copy()

In [34]:
credits = pd.read_csv('data/credits.csv')

In [36]:
credits['cast_'] = credits['cast'].apply(literal_eval)

In [49]:
credits['actors'] = credits['cast_'].apply(lambda x: [d['name'].replace(" ", "") for d in x])

In [77]:
credits_df = credits[['id', 'actors']].copy()

In [94]:
#movies_df.loc[movies_df['id'].isin(['-'])]
movies_df['id'] = movies_df['id'].apply(lambda s: -1 if '-' in s else int(s))

In [78]:
pd.merge([movies_df, keywords_df, credits_df], on='id')

TypeError: merge() missing 1 required positional argument: 'right'

In [96]:
full_df = movies_df.merge(keywords_df, left_on='id', right_on='id')

In [97]:
full_df = full_df.merge(credits_df, on='id')

In [98]:
full_df

,id,runtime,english,keywords__,actors
0,862,81.0,1,"[jealousy, toy, boy, friendship]","[TomHanks, TimAllen, DonRickles, JimVarney, Wa..."
1,8844,104.0,1,"[boardgame, disappearance, basedonchildren'sbo...","[RobinWilliams, JonathanHyde, KirstenDunst, Br..."
2,15602,101.0,1,"[fishing, bestfriend, duringcreditsstinger, ol...","[WalterMatthau, JackLemmon, Ann-Margret, Sophi..."
3,31357,127.0,1,"[basedonnovel, interracialrelationship, single...","[WhitneyHouston, AngelaBassett, LorettaDevine,..."
4,11862,106.0,1,"[baby, midlifecrisis, confidence, aging]","[SteveMartin, DianeKeaton, MartinShort, Kimber..."
...,...,...,...,...,...
46623,439050,90.0,0,[tragiclove],"[LeilaHatami, KouroshTahami, ElhamKorda]"
46624,111109,360.0,0,"[artist, play, pinoy]","[AngelAquino, PerryDizon, HazelOrencio, JoelTo..."
46625,67758,90.0,1,[],"[ErikaEleniak, AdamBaldwin, JulieduPage, James..."
46626,227506,87.0,1,[],"[IwanMosschuchin, NathalieLissenko, PavelPavlo..."


In [106]:
full_df['bag_of_words'] = ''
columns = full_df.columns[-3:-1]
for index, row in full_df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col])+ ' '
    row['bag_of_words'] = words
    
#df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [110]:
full_df_ = full_df[['keywords__', 'actors']].copy()

In [113]:
full_df_['bag_of_words'] = ''
columns = full_df_.columns
for index, row in full_df_.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col])+ ' '
    row['bag_of_words'] = words

In [116]:
full_df = pd.concat([full_df, full_df_], axis=1)

In [121]:
full_df

,id,runtime,english,keywords__,actors,bag_of_words,keywords__,actors,bag_of_words
0,862,81.0,1,"[jealousy, toy, boy, friendship]","[TomHanks, TimAllen, DonRickles, JimVarney, Wa...",,"[jealousy, toy, boy, friendship]","[TomHanks, TimAllen, DonRickles, JimVarney, Wa...",jealousy toy boy friendship TomHanks TimAllen ...
1,8844,104.0,1,"[boardgame, disappearance, basedonchildren'sbo...","[RobinWilliams, JonathanHyde, KirstenDunst, Br...",,"[boardgame, disappearance, basedonchildren'sbo...","[RobinWilliams, JonathanHyde, KirstenDunst, Br...",boardgame disappearance basedonchildren'sbook ...
2,15602,101.0,1,"[fishing, bestfriend, duringcreditsstinger, ol...","[WalterMatthau, JackLemmon, Ann-Margret, Sophi...",,"[fishing, bestfriend, duringcreditsstinger, ol...","[WalterMatthau, JackLemmon, Ann-Margret, Sophi...",fishing bestfriend duringcreditsstinger oldmen...
3,31357,127.0,1,"[basedonnovel, interracialrelationship, single...","[WhitneyHouston, AngelaBassett, LorettaDevine,...",,"[basedonnovel, interracialrelationship, single...","[WhitneyHouston, AngelaBassett, LorettaDevine,...",basedonnovel interracialrelationship singlemot...
4,11862,106.0,1,"[baby, midlifecrisis, confidence, aging]","[SteveMartin, DianeKeaton, MartinShort, Kimber...",,"[baby, midlifecrisis, confidence, aging]","[SteveMartin, DianeKeaton, MartinShort, Kimber...",baby midlifecrisis confidence aging SteveMarti...
...,...,...,...,...,...,...,...,...,...
46623,439050,90.0,0,[tragiclove],"[LeilaHatami, KouroshTahami, ElhamKorda]",,[tragiclove],"[LeilaHatami, KouroshTahami, ElhamKorda]",tragiclove LeilaHatami KouroshTahami ElhamKorda
46624,111109,360.0,0,"[artist, play, pinoy]","[AngelAquino, PerryDizon, HazelOrencio, JoelTo...",,"[artist, play, pinoy]","[AngelAquino, PerryDizon, HazelOrencio, JoelTo...",artist play pinoy AngelAquino PerryDizon Hazel...
46625,67758,90.0,1,[],"[ErikaEleniak, AdamBaldwin, JulieduPage, James...",,[],"[ErikaEleniak, AdamBaldwin, JulieduPage, James...",ErikaEleniak AdamBaldwin JulieduPage JamesRem...
46626,227506,87.0,1,[],"[IwanMosschuchin, NathalieLissenko, PavelPavlo...",,[],"[IwanMosschuchin, NathalieLissenko, PavelPavlo...",IwanMosschuchin NathalieLissenko PavelPavlov ...


In [123]:
full_df = full_df.iloc[:, [0, 1, 2, 8]].copy()

In [124]:
full_df

,id,runtime,english,bag_of_words
0,862,81.0,1,jealousy toy boy friendship TomHanks TimAllen ...
1,8844,104.0,1,boardgame disappearance basedonchildren'sbook ...
2,15602,101.0,1,fishing bestfriend duringcreditsstinger oldmen...
3,31357,127.0,1,basedonnovel interracialrelationship singlemot...
4,11862,106.0,1,baby midlifecrisis confidence aging SteveMarti...
...,...,...,...,...
46623,439050,90.0,0,tragiclove LeilaHatami KouroshTahami ElhamKorda
46624,111109,360.0,0,artist play pinoy AngelAquino PerryDizon Hazel...
46625,67758,90.0,1,ErikaEleniak AdamBaldwin JulieduPage JamesRem...
46626,227506,87.0,1,IwanMosschuchin NathalieLissenko PavelPavlov ...


In [125]:
full_df.to_pickle('bag_of_words.pkl')